In [ ]:
import polars as pl
from datetime import datetime,timedelta
import get_and_store as gas
import polars_indicators as pi
from polars_indicators.indicators import IndicatorResult
from importlib import reload
DATE_COLUNN = "Date"

In [ ]:
reload(pi)
reload(pi.strategies.buy_x_week_low)

In [ ]:
symbols = ['A','AA']

In [ ]:
#df = gas.get_symbols_df(symbols)

In [ ]:
try:
    df = saved_df.clone()
except NameError:
        df = gas.get_all_saved_data_df()
        df = df.filter(pl.col('Date') >= datetime(2020, 1, 1))
        df = df.collect()

In [ ]:
saved_df = df.clone()

In [ ]:
df = df.with_columns(pl.col(pi.indicators.DATE_COLUMN).cast(pl.Datetime))
df

In [ ]:
lookback = timedelta(weeks=52)
modresult = pi.strategies.buy_x_week_low.strategy(df, lookback)

In [ ]:
min52 = "52_week_min"
df = df.with_columns(pl.col("Low").rolling_min('52w', by=pi.indicators.DATE_COLUMN).over(pi.indicators.SYMBOL_COLUMN).alias(min52))

In [ ]:
df = df.filter(pl.col(pi.indicators.DATE_COLUMN) > datetime(2021, 1, 1)) #this gets a a proper min over the last year even though we're not trading that time

In [ ]:
df = df.filter(pl.col(pi.indicators.CLOSE_COLUMN).mean().over(pi.indicators.SYMBOL_COLUMN) > 10)

In [ ]:
target = pi.indicators.targeted_value(df, min52)
df = target.df
df

In [ ]:
percentage = -5
p_stop = pi.indicators.entry_percentage_stop(df, percentage, target.column)
df = p_stop.df
df.filter(pl.col(p_stop.column).is_not_null())

In [ ]:
trail = pi.indicators.trailing_stop(df, bars=2)

In [ ]:
eod = pi.indicators.end_of_data_stop(trail.df)
df = eod.df

In [ ]:
enter_column = target.column

In [ ]:
#clear out trailing stop values on entry columns since that's handled by percentage stop
df = df.with_columns(pl.when(pl.col(enter_column).is_not_null()).then(pl.lit(None)).otherwise(pl.col(trail.column)).alias(trail.column))

In [ ]:
exit_column = "exit_column"
df = df.with_columns(pl.coalesce(pl.col(p_stop.column),pl.col(trail.column), pl.col(eod.column)).alias(exit_column))
df

In [ ]:
#limit = pi.limit_entries(trail.df, 2, trail.column)

In [ ]:
ret = pi.indicators.create_trade_ids(df, enter_column, exit_column)

In [ ]:
summ = pi.indicators.summarize_trades(ret.df, ret.column, enter_column, exit_column, [])
summ

In [ ]:
summ.describe()

In [ ]:
sma_up = "sma_up" #left off here. Need to validate if sma filter is "cheating" by looking at the sma result after the close of the current bar
sma100 = pi.indicators.simple_moving_average(modresult.df, 100)
moddf = sma100.df.with_columns((pl.col(sma100.column) > pl.col(sma100.column).shift(1)).alias(sma_up))
moddf.filter(pl.col(sma_up) != pl.col(sma_up).shift() | (pl.col(sma_up) != pl.col(sma_up).shift(-1))).select(pi.indicators.SYMBOL_COLUMN,"Date", sma_up, sma100.column)

In [ ]:
summ.select(pl.col("Gain/Loss%").sum(),
            (pl.col("Gain/Loss%") > 0).sum().alias("Winngers"),
            (pl.col("Gain/Loss%") < 0).sum().alias("Losers"),
            pl.col(pi.indicators.SYMBOL_COLUMN).count().alias("Trades"),
            pl.col("Gain/Loss%").mean().alias("Average_Gain/Loss%"),
            pl.col("Length").mean().alias("Average_Length"),
            pl.col("Highest%").max(),
            pl.col("Lowest%").min()
            )